In [ ]:
# Import statements
import datetime
import pandas as pd
import numpy as np
import os
from tqdm import tqdm 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
import warnings # Supress all warnings
warnings.filterwarnings("ignore")
from concurrent.futures import ProcessPoolExecutor
import lightgbm as lgb
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from joblib import Parallel, delayed
import glob
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# -------------------------------- # 
# DATA LOADING AFTER FACTOR SELECTION 
# -------------------------------- #

print(datetime.datetime.now())
pd.set_option("mode.chained_assignment", None)

work_dir = "/teamspace/studios/this_studio/"

# Load factor list
factor_csv_path = os.path.join(work_dir, "Quant_Data_Labels/selected_factors.csv") #changed from factor_char_list.csv to selected_factors.csv
factor_list_raw = pd.read_csv(factor_csv_path)
factor_list = factor_list_raw["factor"].tolist() if "factor" in factor_list_raw.columns else factor_list_raw.iloc[:, 0].tolist() #column for csv is variable. Changed variable to factor as that is what is used in selected_factors

# Determine which factors exist in Parquet
file_path = os.path.join(work_dir, "Actual_Data/ret_sample.parquet")
parquet_cols = pd.read_parquet(file_path, columns=None).columns.tolist()
factors_to_load = [f for f in factor_list if f in parquet_cols]

# Load only needed columns
cols_to_load = ["date", "gvkey","ret_eom","year","month","stock_ret","id"] + factors_to_load #added in more identifiers
data = pd.read_parquet(file_path, columns=cols_to_load)
data = data[data["stock_ret"].notna()]
stock_vars = factors_to_load
ret_var = "stock_ret"

# Monthly scaling & ranking
monthly = data.groupby("date")

def process_month(group_tuple):
    _, group = group_tuple

    # Fill missing with monthly median
    medians = group[stock_vars].median(skipna=True)
    group[stock_vars] = group[stock_vars].fillna(medians)

    # Rank variables
    group[stock_vars] = group[stock_vars].rank(method="dense") - 1

    # Scale to [-1, 1]
    for stock in stock_vars:
        max_val = group[stock].max()
        group[stock] = (group[stock] / max_val) * 2 - 1 if max_val > 0 else 0

    return group

with ProcessPoolExecutor() as executor:
    result = list(tqdm(executor.map(process_month, monthly), total=len(monthly)))

data = pd.concat(result, ignore_index=True) #ignores original index and creates new index
data["date"] = pd.to_datetime(data["date"], format="%Y%m%d")

print("Data loaded and preprocessed:", data.shape)

In [ ]:
# -------------------------------- # 
# DATA LOADING (BEFORE FACTOR SELECTION) 
# -------------------------------- #
# for timing purpose
print(datetime.datetime.now())
# Supress numpy runtime warnings
# turn off pandas Setting with Copy Warning
pd.set_option("mode.chained_assignment", None)

# set working directory
work_dir = "/teamspace/studios/this_studio/"

# read sample data
file_path = os.path.join(
    work_dir, "Actual_Data/ret_sample.parquet"
)  # replace with the correct file name
# print(os.path.exists(file_path))

raw = pd.read_parquet(
    file_path 
) # the date is the first day of the return month (t+1)




# read list of predictors for stocks
file_path = os.path.join(
    work_dir, "Quant_Data_Labels/factor_char_list.csv"
)  # replace with the correct file name
stock_vars = list(pd.read_csv(file_path)["variable"].values)
# define the left hand side variable
ret_var = "stock_ret" #not normailzing the target on the predictive factors!!!
new_set = raw[
    raw[ret_var].notna()
].copy()  # create a copy of the data and make sure the left hand side is not missing

# transform each variable in each month to the same scale
monthly = new_set.groupby("date")
data = pd.DataFrame()

def parallel(groups):
    _,group = groups
    group = group.copy()
    
    var_median = group[stock_vars].median(skipna=True)
    group[stock_vars] = group[stock_vars].fillna(var_median)

    group[stock_vars] = group[stock_vars].rank(method="dense")-1
    
    for stock in stock_vars:
        stock_max = group[stock].max()
        if stock_max > 0:
            group[stock] = (group[stock] / stock_max) * 2 - 1
        else:
            group[stock] = 0  # in case of all missing values
            #print("Warning:", date[stock], stock, "set to zero.") # Print command
    return group


with ProcessPoolExecutor(max_workers=32) as executor:

    result = list(tqdm(executor.map(parallel, monthly), total=len(monthly)))
    
data = pd.concat(result, ignore_index = True) #ignores original index and creates new index

# Convert format of the date to a Timestamp
data["date"] = pd.to_datetime(data["date"], format="%Y%m%d")


In [ ]:
# -----------------------------
# Factor selection (once for full dataset)
# -----------------------------

# -----------------------------
# Load factor list from CSV
# -----------------------------
factor_csv_path = os.path.join(work_dir, "Quant_Data_Labels/factor_char_list.csv")
factor_list_raw = pd.read_csv(factor_csv_path)
# If the CSV has a column header, use it; otherwise fallback to first column
if "factor" in factor_list_raw.columns:
    factor_list = factor_list_raw["factor"].tolist()
else:
    factor_list = factor_list_raw.iloc[:, 0].tolist()

def select_factors(data, factor_list, ret_var, missing_thresh=0.5,
                   corr_thresh=0.95, lgb_n_estimators=100,
                   corr_sample_size=50000, lgb_sample_size=50000,
                   random_state=42):
    # -----------------------------
    # 1. Clean factor list
    # -----------------------------
    factors_clean = [f for f in factor_list if f in data.columns]
    if not factors_clean:
        print("No factors found in data.")
        return []

    # -----------------------------
    # 2. Remove highly correlated factors
    # -----------------------------
    n_corr_sample = min(corr_sample_size, len(data))
    corr_idx = np.random.RandomState(random_state).choice(data.index, size=n_corr_sample, replace=False)
    X_corr_sample = data.loc[corr_idx, factors_clean].fillna(0)
    corr_matrix = X_corr_sample.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [c for c in upper.columns if any(upper[c] > corr_thresh)]
    factors_corr = [f for f in factors_clean if f not in to_drop]
    print(f"{len(factors_corr)} factors remain after removing highly correlated factors (>{corr_thresh})")
    if not factors_corr:
        return []

    # -----------------------------
    # 3. Sample for LightGBM
    # -----------------------------
    n_lgb_sample = min(lgb_sample_size, len(data))
    lgb_idx = np.random.RandomState(random_state + 1).choice(data.index, size=n_lgb_sample, replace=False)
    X_lgb_sample = data.loc[lgb_idx, factors_corr].fillna(0)
    Y_raw = pd.to_numeric(data.loc[lgb_idx, ret_var], errors="coerce").fillna(0)

    if Y_raw.nunique() <= 1:
        print("Target is constant. Skipping LightGBM.")
        return factors_corr  # fallback

    Y_bin = (Y_raw > 0).astype(int)

    # -----------------------------
    # 4. LightGBM feature importance
    # -----------------------------
    lgb_model = lgb.LGBMClassifier(
        n_estimators=lgb_n_estimators,
        n_jobs=-1,
        verbose=10
    )
    lgb_model.fit(X_lgb_sample, Y_bin)
    factors_final = [f for f, imp in zip(factors_corr, lgb_model.feature_importances_) if imp > 0]
    print(f"{len(factors_final)} factors selected by LightGBM importance (>0)")

    return factors_final


# -----------------------------
# Precompute factors and save
# -----------------------------
full_train = data[data["date"] < pd.to_datetime("20250101")]
selected_factors = select_factors(full_train, factor_list, ret_var="stock_ret", corr_thresh=0.75) 
print(f"Precomputed factors: {len(selected_factors)}")

factors_file = os.path.join(work_dir, "Quant_Data_Labels/selected_factors.csv")
# os.makedirs(os.path.dirname(factors_file), exist_ok=True)
pd.DataFrame(selected_factors, columns=["factor"]).to_csv(factors_file, index=False)
print(f"Factors saved to {factors_file}")

In [ ]:
# -----------------------------
# Transformer classifier
# -----------------------------
class SimpleTransformerClassifier(nn.Module):
    def __init__(self, feature_dim, n_heads=4, hidden_dim=64, n_layers=2, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(feature_dim, hidden_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=n_heads,
            dim_feedforward=hidden_dim*4,
            dropout=dropout,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.cls_head = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        h = self.input_proj(x)
        h = self.encoder(h)
        h = h.mean(dim=1)  # mean pooling over sequence
        logit = self.cls_head(h)
        return torch.sigmoid(logit)


IMPLEMENTING finBERT LLM Code

In [ ]:


# -----------------------------
# SETTINGS
# -----------------------------
work_dir = "/teamspace/studios/this_studio/"
PKL_DIR = os.path.join(work_dir, "Text_Data_picklefiles")
LINK_CSV = os.path.join(work_dir, "Quant_Data_Labels/cik_gvkey_linktable_USA_only.csv")
CACHE_DIR = os.path.join(PKL_DIR, "finbert_forward_by_gvkey_fast")
complete_parquet = os.path.join(work_dir, "Actual_Data/risk_score_256.parquet")
os.makedirs(CACHE_DIR, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL = "ProsusAI/finbert"
MAX_LEN = 256        # 256 tokens per filing --> may need to increase this as probably truncating long text fillings
BATCH_TXT = 128       
PRINT_EVERY = 10000    # progress prints
complete_parquet = os.path.join(work_dir, f"Actual_Data/risk_score_{MAX_LEN}.parquet")

# -----------------------------
# LOAD FAST TOKENIZER & MODEL
# -----------------------------
tok = AutoTokenizer.from_pretrained(MODEL, use_fast=True)
mdl = AutoModelForSequenceClassification.from_pretrained(MODEL).to(DEVICE).eval()
try:
    mdl.half()  # converting to 16bit for memory purposes
except:
    pass

@torch.no_grad()
def fast_score_texts(texts):
    
    enc = tok(texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt").to(DEVICE)
    enc = {k: v.to(dtype=torch.int64) if k=="input_ids" else v for k,v in enc.items()}
    logits = mdl(**enc).logits.half()  # FP16
    probs = torch.softmax(logits, dim=-1).cpu().numpy()
    return probs[:,0] - probs[:,2]  # risk = p_neg - p_pos

# -----------------------------
# LOAD LINK TABLE
# -----------------------------
def load_link_table(path):
    lt = pd.read_csv(path)
    lt.columns = [c.lower() for c in lt.columns]
    lt["cik"] = lt["cik"].astype(str).str.strip()
    lt["gvkey"] = lt["gvkey"].astype(str).str.replace(r"\.0$","",regex=True).str.strip()
    lt["linkdt"] = pd.NaT #assuming each gvkey and cik are unique
    lt["linkenddt"] = pd.NaT
    return lt[["cik","gvkey","linkdt","linkenddt"]].copy()

link = load_link_table(LINK_CSV)

# -----------------------------
# PROCESS FILINGS YEAR-BY-YEAR
# -----------------------------
def build_finbert_by_year(pkl_dir=PKL_DIR, link_table=link, cache_dir=CACHE_DIR):
    all_files = sorted(glob.glob(os.path.join(pkl_dir, "text_us_*.pkl")))
    all_years_data = []

    for fp in all_files:
        year = os.path.basename(fp).split("_")[2].split(".")[0]
        cache_file = os.path.join(cache_dir, f"finbert_{year}.parquet")
        if os.path.exists(cache_file): #saves cached files and loads them seperately which is good due to maximum session run times.
            print(f"Year {year} cached, loading...")
            all_years_data.append(pd.read_parquet(cache_file))
            continue

        df0 = pd.read_pickle(fp)
        if "date" not in df0 or "cik" not in df0:
            continue

        df = pd.DataFrame({
            "cik": df0["cik"].astype(str).str.strip(),
            "filing_date": pd.to_datetime(df0["date"].astype(str), format="%Y%m%d", errors="coerce"),
            "text": (df0.get("mgmt","").fillna("") + " " + df0.get("rf","").fillna("")).str.replace(r"\s+"," ",regex=True).str.strip()
        }).dropna(subset=["cik","filing_date"])
        df = df[df["text"].str.len() > 0].reset_index(drop=True)

        # Batch inference
        risks = []
        n_batches = int(np.ceil(len(df) / BATCH_TXT))
        for i in range(n_batches):
            batch_texts = df["text"].iloc[i*BATCH_TXT:(i+1)*BATCH_TXT].tolist()
            risks.extend(fast_score_texts(batch_texts))
            if i % (PRINT_EVERY // BATCH_TXT) == 0:
                print(f"Year {year}: batch {i+1}/{n_batches} ({len(risks)}/{len(df)}) filings scored")

        df["finbert_risk"] = np.array(risks, dtype=float)

        # Merge with gvkey
        m = df.merge(link_table, on="cik", how="left")
        m = m.dropna(subset=["gvkey"])
        m = m.sort_values(["cik","filing_date"]).drop_duplicates(["cik","filing_date"], keep="last")

        # Monthly aggregation. Not forward aligning using change from previous months data
        m["year"] = m["filing_date"].dt.year
        m["month"] = m["filing_date"].dt.month
        risk_m = m.groupby(["gvkey","year","month"], as_index=False)["finbert_risk"].mean()
        risk_m["finbert_risk_mom"] = risk_m.groupby("gvkey")["finbert_risk"].diff()
     
        # Cache
        risk_m.to_parquet(cache_file, index=False) 
        all_years_data.append(risk_fwd)
        print(f"Year {year} done. Saved to {cache_file}")


    return pd.concat(all_years_data, ignore_index=True)

# ----------------------------------------
# RUN FINBERT SCORING AND CREATE RISK FILE
# ----------------------------------------
risk = build_finbert_by_year()

# -----------------------------
# MERGE WITH MAIN DATA
# -----------------------------
data = data.merge(
    risk,
    on=["gvkey","year","month"],
    how="left"
)

#section adds an indicator to differentiate between having a zero sentiment to being empty. The original code just fillna(0.0) for everything that is empty/NaN
data["has_filing"] = (~data[["finbert_risk_fwd", "finbert_risk_mom_fwd"]].isna().any(axis=1)).astype(int)

data["finbert_risk_fwd"] = data["finbert_risk_fwd"].where(data["has_filing"]==1, 0.0)
data["finbert_risk_mom_fwd"] = data["finbert_risk_mom_fwd"].where(data["has_filing"]==1, 0.0)


# Add to factor list
for c in ["finbert_risk_fwd", "finbert_risk_mom_fwd", "has_filing"]:
    if c not in factor_list:
        factor_list.append(c)






In [ ]:

# -----------------------------
# Expanding-window prediction
# -----------------------------
starting = pd.to_datetime("20050101", format="%Y%m%d")
counter = 0
pred_out = pd.DataFrame()
ret_var = "stock_ret"

# Hyperparameters
BATCH_SIZE = 256 #increase BATCH_SIZE as appropriate for system used. Multiple GPUs can handle a higher BATCH_SIZE which allows for faster runtime
EPOCHS = 10
LR = 1e-3
window_train_years = 8
window_val_years = 2

while (starting + pd.DateOffset(years=window_train_years + window_val_years + 1 + counter)) <= pd.to_datetime("20260101"):
    cutoff_train_start = starting
    cutoff_train_end = starting + pd.DateOffset(years=window_train_years + counter)
    cutoff_val_end = cutoff_train_end + pd.DateOffset(years=window_val_years)
    cutoff_test_end = cutoff_val_end + pd.DateOffset(years=1)

    train = data[(data["date"] >= cutoff_train_start) & (data["date"] < cutoff_train_end)]
    validate = data[(data["date"] >= cutoff_train_end) & (data["date"] < cutoff_val_end)]
    test = data[(data["date"] >= cutoff_val_end) & (data["date"] < cutoff_test_end)]
# -----------------------------
    # Standardize features
    # -----------------------------
    # Assuming 'data' already contains only useful factors
    factors_in_data = [f for f in factor_list if f in data.columns]
    scaler = StandardScaler().fit(train[factors_in_data])
    X_train = scaler.transform(train[factors_in_data])
    X_val = scaler.transform(validate[factors_in_data])
    X_test = scaler.transform(test[factors_in_data])

    # -----------------------------
    # Prepare classification target
    # -----------------------------
    Y_train_bin = (train[ret_var].values > 0).astype(int) #change all of these from ret_1_0 to stock_ret
    Y_val_bin = (validate[ret_var].values > 0).astype(int)
    Y_test_bin = (test[ret_var].values > 0).astype(int)

    # -----------------------------
    # Transformer classifier
    # -----------------------------
    X_train_t = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
    X_val_t = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1)
    X_test_t = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
    Y_train_t = torch.tensor(Y_train_bin, dtype=torch.float32).unsqueeze(1)

    train_dataset = TensorDataset(X_train_t, Y_train_t)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SimpleTransformerClassifier(feature_dim=len(factors_in_data)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.BCELoss()

    model.train()
    for epoch in range(EPOCHS):
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            prob = model(xb)
            loss = criterion(prob, yb)
            loss.backward()
            optimizer.step()

    # -----------------------------
    # Inference + isotonic calibration
    # -----------------------------
    model.eval()
    with torch.no_grad():
        val_probs_raw = model(X_val_t.to(device)).cpu().numpy().flatten()
        test_probs_raw = model(X_test_t.to(device)).cpu().numpy().flatten()

    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(val_probs_raw, Y_val_bin)
    test_probs_cal = iso.predict(test_probs_raw)

    # -----------------------------
    # Conditional expected return prediction
    # -----------------------------
    X_train_pos = X_train[train[ret_var] > 0] #previously ret_1_0 for all of these
    Y_train_pos = train.loc[train[ret_var] > 0, ret_var].values
    X_train_neg = X_train[train[ret_var] <= 0]
    Y_train_neg = train.loc[train[ret_var] <= 0, ret_var].values

    lgb_pos = lgb.LGBMRegressor()
    lgb_pos.fit(X_train_pos, Y_train_pos)
    lgb_neg = lgb.LGBMRegressor()
    lgb_neg.fit(X_train_neg, Y_train_neg)

    mu_test = test_probs_cal * lgb_pos.predict(X_test) + \
              (1 - test_probs_cal) * lgb_neg.predict(X_test)

    # -----------------------------
    # Store outputs
    # -----------------------------
    reg_pred = test[["year", "month", "stock_ret", "id", "stock_ret"]].copy() #switched ret_1_0 to stock_ret. ID not in index.
    reg_pred["transformer_prob_raw"] = test_probs_raw
    reg_pred["transformer_prob_cal"] = test_probs_cal
    reg_pred["exp_ret"] = mu_test
    pred_out = pred_out._append(reg_pred, ignore_index=True)
    counter += 1
    print(f"Window {counter} done, {len(factors_in_data)} factors used.") 

# -----------------------------
# Output results
# -----------------------------
pred_out.to_csv("output_transformer_lgb_precomputed_factors.csv", index=False)
print("done!")

# -----------------------------
# OOS R2 -> Model predictivity
# -----------------------------

yreal = pred_out[ret_var].to_numpy().flatten() #previously ret_1_0
ypred = pred_out["exp_ret"].to_numpy().flatten()

oos_r2 = 1 - np.sum((yreal - ypred)**2) / np.sum(yreal**2)

print("OOS R^2:", oos_r2)


PORTFOLIO OPTIMIZATION CODE FOLLOWING:

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------------------------------
# Mean Variance Portfolio Optimization - Markowitz portfolio optimization using cvxpy library for convex optimization problems (not including monthly turnover)
# -------------------------------------------------------------------------------------------------------------------------------------------------------------

#this method is instead of splitting the stocks into 10 portfolios and ranking them. Shorting the lowest and long the highest. This is providing weights to each stock and then choosing the optimal porfolio

import cvxpy as cp
import statsmodels.formula.api as sm

In [ ]:
# ---------------------------
# 1. Prepare predicted returns
# ---------------------------

pred_returns = pred_out.pivot(index="date", columns="id", values="exp_ret").fillna(0)

pred_returns["year_month"] = pred_returns["date"].dt.to_period("M")

pred_ret_monthly = pred_returns.groupby("year_month")

mu = np.array(pred_returns.mean())

covMatrix = pred_returns.cov().values

noa = pred_returns.shape[1] #determining the number of stocks

In [ ]:
# ---------------------------
# 2. Set up optimization problem
# ---------------------------

w = cp.Variable(noa) #creating a variable for the weights 

gamma = 3 #Higher gamma meens more risk adverse. Lower gamma meens taking on more risk

ret = mu @ w
risk = cp.quad_form(w, covMatrix)

goal = cp.Maximize((ret - risk*gamma))

constraints = [ #can add constraint w>=0 would give a long only portfolio. Maybe look into different possible constraints that could make the optimization better
    cp.sum(w) == 1 #fully invested
]

problem = cp.Problem(goal, constraints) #not sure why I am getting error with constraints type but I think it is just because files are not loaded
problem.solve(solver=cp.SCS)

#weights_opt = pd.DataFrame( Adding weights to the dataframe pred_returns. May be useful.
   # w.value, index=pred_returns.columns, columns=["Weight"]
#)
#print(weights_opt.round(4))

In [ ]:
# -----------------------------------------------
# 3. Compute portfolio returns performace metrics
# -----------------------------------------------

portfolio_returns = np.dot(pred_returns.values, w.value)

cum_ret = (1+portfolio_returns).cumprod()-1

mkt_csv_path = os.path.join(work_dir, "Quant_Data_Labels/mkt_factor.csv")

portfolio_df = pd.DataFrame({
    "date": pred_returns.index,
    "portfolio": portfolio_returns
})

portfolio_df['year'] = portfolio_df['date'].dt.year #extracting the year
portfolio_df['month'] = portfolio_df['date'].dt.month #extracting the date



def SharpeRatio(returns):
    return returns.mean() / returns.std() * np.sqrt(12)

def maxDrawDown(cum_returns):
    running_max = np.maximum.accumulate(cum_returns + 1)
    dd = (cum_returns + 1) / running_max - 1
    return dd.max()

def annualizedReturn(returns):
    return returns.mean()*12 #multiply the average monthly returns by 12 to get the average yearly return

def anualizedStd(returns):
    return returns.std() * 12

def maxmonthLoss(returns):
    return returns.min() #np.min(returns)

def annualizedAlphaInfoTstat(returns_df, file_path):
    mkt = pd.read_csv(file_path)

    mkt['year'] = mkt['year'].astype(int)
    mkt['month'] = mkt['month'].astype(int)

    merged_df = returns_df.merge(mkt, how="inner", on=["year", "month"])

    merged_df['port_excess'] = merged_df['portfolio'] - merged_df['rf']

    nw_ols = sm.ols(formula="port_excess ~ mkt_rf", data=merged_df).fit(
    cov_type="HAC", cov_kwds={"maxlags": 3}, use_t=True
    )
    
    alpha = nw_ols.parms["Intercept"] * 12 #annualized alpha
    
    t_statistic = nw_ols.tvales["Intercept"] #t statistic 
    
    Info_Ratio = alpha/(np.sqrt(nw_ols.mse_resid) * np.sqrt(12)) #annualized info_ratio

    return alpha,t_statistic,Info_Ratio
    
    


Sharpe_Annualized = SharpeRatio(portfolio_returns)
max_drawdown = maxDrawDown(cum_ret)
annual_return = annualizedReturn(portfolio_returns)
annaul_std = anualizedStd(portfolio_returns)
max_loss = maxmonthLoss(portfolio_returns)
annualized_data = annualizedAlphaInfoTstat(portfolio_df, mkt_csv_path)

print(f"The annualized Sharpe ratio is {Sharpe_Annualized:.4f} and the maximum drawdown is {max_drawdown:.4f}")



In [ ]:
# -----------------------------------------------------------
# Portfolio optimization including monthly portfolio turnover
# -----------------------------------------------------------

import cvxpy as cp
import statsmodels.formula.api as sm

# -----------------------------------------------
# 1. Prepare data
# -----------------------------------------------

pred_returns = pred_out.pivot(index="date", columns="id", values="exp_ret").fillna(0)
pred_returns.index = pd.to_datetime(pred_returns.index)
pred_returns["year_month"] = pred_returns.index.to_period("M")

# -----------------------------------------------
# 2. Setting up problem
# -----------------------------------------------

gamma = 3
weights_prev = None
portfolio_history = []
turnover_list = []
dates_list = []

for ym, month_data in pred_returns.groupby("year_month"):
    month_data = month_data.drop(columns="year_month")
    mu = month_data.mean().values
    covMatrix = month_data.cov().values
    noa = month_data.shape[1]

    w = cp.Variable(noa)
    ret = mu @ w
    risk = cp.quad_form(w, covMatrix)
    objective = cp.Maximize(ret - gamma * risk)
    constraints = [cp.sum(w) == 1] #fully invested. Not long only.

    if weights_prev is not None:
        turnover = cp.norm1(w - weights_prev)
        constraints.append(turnover <= 0.2)

    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS, verbose=False)

    if w.value is None:
        print(f"Optimization failed for {ym}")
        continue

    weights = np.array(w.value)
    portfolio_history.append(weights)
    dates_list.append(ym)

    if weights_prev is not None:
        monthly_turnover = np.sum(np.abs(weights - weights_prev))
        turnover_list.append(monthly_turnover)
    else:
        turnover_list.append(np.nan)

    weights_prev = weights

weights_df = pd.DataFrame(portfolio_history, index=dates_list, columns=month_data.columns)
turnover_series = pd.Series(turnover_list, index=dates_list, name="turnover")

# -----------------------------------------------
# 3. Compute monthly returns
# -----------------------------------------------

portfolio_returns = (pred_returns.drop(columns="year_month") * weights_df.shift(1)).sum(axis=1)
portfolio_returns = portfolio_returns.groupby(pred_returns["year_month"]).mean()
cum_ret = (1 + portfolio_returns).cumprod() - 1

portfolio_df = pd.DataFrame({
    "year_month": portfolio_returns.index.astype(str),
    "portfolio": portfolio_returns.values
})
portfolio_df["year"] = portfolio_df["year_month"].str[:4].astype(int)
portfolio_df["month"] = portfolio_df["year_month"].str[5:].astype(int)

# -----------------------------------------------
# 4. Calculations of performance metrics
# -----------------------------------------------

def SharpeRatio(returns):
    return returns.mean() / returns.std() * np.sqrt(12)

def maxDrawDown(cum_returns):
    running_max = np.maximum.accumulate(cum_returns + 1)
    dd = (cum_returns + 1) / running_max - 1
    return dd.min()  # most negative drawdown

def annualizedReturn(returns):
    return returns.mean() * 12

def annualizedStd(returns):
    return returns.std() * np.sqrt(12)

def maxmonthLoss(returns):
    return returns.min()

def annualizedAlphaInfoTstat(returns_df, file_path):
    mkt = pd.read_csv(file_path)
    mkt['year'] = mkt['year'].astype(int)
    mkt['month'] = mkt['month'].astype(int)
    merged_df = returns_df.merge(mkt, how="inner", on=["year", "month"])
    merged_df['port_excess'] = merged_df['portfolio'] - merged_df['rf']

    nw_ols = sm.ols(formula="port_excess ~ mkt_rf", data=merged_df).fit(
        cov_type="HAC", cov_kwds={"maxlags": 3}, use_t=True
    )

    alpha = nw_ols.params["Intercept"] * 12  # annualized alpha
    t_statistic = nw_ols.tvalues["Intercept"]
    info_ratio = alpha / (np.sqrt(nw_ols.mse_resid) * np.sqrt(12))

    return alpha, t_statistic, info_ratio

# compute metrics
Sharpe_Annualized = SharpeRatio(portfolio_returns)
max_drawdown = maxDrawDown(cum_ret)
annual_return = annualizedReturn(portfolio_returns)
annual_std = annualizedStd(portfolio_returns)
max_loss = maxmonthLoss(portfolio_returns)
avg_turnover = np.nanmean(turnover_series)

mkt_csv_path = os.path.join(work_dir, "Quant_Data_Labels/mkt_factor.csv")
alpha, t_stat, info_ratio = annualizedAlphaInfoTstat(portfolio_df, mkt_csv_path)

# -----------------------------------------------
# 5. Results
# -----------------------------------------------

print(f"Annualized Sharpe ratio: {Sharpe_Annualized:.3f}")
print(f"Annualized return: {annual_return:.3%}")
print(f"Annualized volatility: {annual_std:.3%}")
print(f"Maximum drawdown: {max_drawdown:.3%}")
print(f"Maximum monthly loss: {max_loss:.3%}")
print(f"Average monthly turnover: {avg_turnover:.3%}")
print(f"Annualized alpha: {alpha:.3%}, t-stat: {t_stat:.2f}, Info ratio: {info_ratio:.3f}")


In [ ]:
def timestamp_to_int(timestamp):
    """Convert timestamp to integer format YYYYMMDD"""
    if isinstance(timestamp, pd.Timestamp):
        dt = timestamp.to_pydatetime()
    elif isinstance(timestamp, datetime):
        dt = timestamp
    else:
        # if it's a string, parse it first
        dt = pd.to_datetime(timestamp)
    
    # format as YYYYMMDD integer
    return int(dt.strftime('%Y%m%d'))